In [1]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *

#np.set_printoptions(precision=4)
import math
from utils import *

nrows = 8
nS = nrows**2
nA = 4
seed = get_current_seed()
gamma = .9
tau = 1.
#tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
#mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)

env = FrozenLakeEnv(is_slippery=False, seed=seed, desc=generate_random_map(nrows))#, render_mode="human")
xi_frozen = np.ones(env.nS) * 1/env.nS
env.reset()

Current seed for result reproducibility: 132958061972956381563039042393515089139


(0, {'prob': 1})

In [2]:
tmdp = TMDP(env, xi_frozen, tau=tau, gamma=gamma, seed=seed)

In [3]:
status_step = 500000
episodes = 500000
batch_nS = 1
temperature = 20

In [4]:
done = False
Q = np.zeros((tmdp.nS, tmdp.nA))
tau = 1.
while not done:
    if tau == 0:
        done = True
    batch_res = batch_q_learning(tmdp, Q, alpha=.25, episodes=episodes, status_step=status_step, batch_nS=batch_nS)    
    Q = batch_res["Qs"][-1]
    tau = max(0.,tau-0.05)
    tmdp.update_tau(tau)
    print("Updating tau to {}".format(tau))

Updating tau to 0.95


KeyboardInterrupt: 

In [ ]:
from utils import *
res = {"Q":Q}
json_res = to_json(res, indent=True)
write_results(nrows, nA, "FrozenLake", json_res, "w")
json_res = read_results(nrows, nA, "FrozenLake")
res = from_json(json_res)

In [ ]:
Q = np.array(res["Q"])
tmdp = TMDP(env, xi_frozen, tau=0., gamma=gamma, seed=seed)
env.render_mode = "human"
env.is_slippery = False
tmdp.reset()
done = False
step = 0
while True:
    a = greedy(tmdp.env.s, Q, tmdp.env.allowed_actions[int(tmdp.env.s)])
    s_prime, reward, flags, prob = tmdp.step(a)
    if flags["done"]:
        tmdp.reset()
    step +=1
    if step > max(1000,nrows*2):
        break